In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

path = 'return-data/'
filelist = os.listdir(path) 
dfs = [pd.read_json(path+file, lines=True) for file in filelist]
toydata = pd.concat(dfs, ignore_index=True)

In [2]:
toydata = toydata.merge(toydata['basket'].apply(lambda x: pd.Series({'c_0': x.count(0), 'c_1': x.count(1),'c_2': x.count(2),'c_3': x.count(3),'c_4': x.count(4),'c_5': x.count(5)})), left_index=True, right_index=True)
toydata.drop(['basket','transactionId','zipCode'],axis=1,inplace=True)
toydata.head()

,returnLabel,totalAmount,c_0,c_1,c_2,c_3,c_4,c_5
0,0,264,1,3,3,0,0,1
1,1,60,0,1,1,1,3,0
2,0,60,0,0,0,0,0,1
3,0,333,1,0,0,1,1,0
4,0,378,2,1,2,1,0,0


In [3]:
def undersample_binarydata(dataset, label):
    #value_counts always returns most common value first
    most_common_value = dataset[label].value_counts().index.tolist()[0]
    least_common_value = dataset[label].value_counts().index.tolist()[1]
    
    number_of_rows_to_remove = len(dataset[dataset[label] == most_common_value]) - len(dataset[dataset[label] == least_common_value])
    dataset_most_common = dataset[dataset[label] == most_common_value] 
    drop_rows = np.random.choice(dataset_most_common.index, number_of_rows_to_remove, replace=False)

    dataset_undersampled = dataset.drop(drop_rows)
    
    return dataset_undersampled

In [4]:
toydata_undersample = undersample_binarydata(toydata, "returnLabel")

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(toydata.drop('returnLabel',axis=1),toydata['returnLabel'], test_size=0.30,random_state=101)
X_train_under, X_test_under, y_train_under, y_test_under = train_test_split(toydata_undersample.drop('returnLabel',axis=1),toydata_undersample['returnLabel'], test_size=0.30,random_state=101)

In [6]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)

clf_under = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf_under.fit(X_train_under,y_train_under)
predictions_under = clf_under.predict(X_test)

In [7]:
from sklearn.metrics import accuracy_score

print("Accuracy:",accuracy_score(y_test, predictions))
print("Accuracy Undersampling:",accuracy_score(y_test, predictions_under))

('Accuracy:', 0.9678494623655914)
('Accuracy Undersampling:', 0.6516129032258065)
